In [1]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/temp.txt', sep='\t', header=None)
df2 = pd.read_csv('/gscmnt/gc2508/dinglab/estorrs/rnaediting/CPTAC3.RNA-Seq.TA.samples.txt', sep='\t', header=None)

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df1.columns = ['sample_id', 'sample', 2,3,4,'fp',6,7,8,'uuid',10]
df2.columns = ['sample']

In [ ]:
df1.head()

In [ ]:
merged = df2.merge(df1, on='sample')

In [ ]:
merged.head()

In [ ]:
merged.shape

In [ ]:
df2.shape

In [ ]:
len(set(merged['sample']))

In [ ]:
merged = merged[['sample', 'sample_id', 'fp', 'uuid']]

In [ ]:
merged.head()

In [ ]:
merged.to_csv('/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/mgi.cptac3.rna-seq.ta.samples.tsv', sep='\t',
             header=None, index=None)

In [ ]:
df1 = pd.read_csv('/gscmnt/gc2508/dinglab/estorrs/rnaediting/katmai/temp.txt', sep='\t', header=None)
df1.columns = ['sample_id', 'sample', 2,3,4,'fp',6,7,8,'uuid',10]


In [ ]:
katmai_merged = df2.merge(df1, on='sample')

In [ ]:
len(set(katmai_merged['sample']))

In [ ]:
set(katmai_merged['sample']) - set(merged['sample'])

nothing on katmai that's not on mgi

In [ ]:
df1 = pd.read_csv('/gscmnt/gc2508/dinglab/estorrs/rnaediting/denali/temp.txt', sep='\t', header=None)
df1.columns = ['sample_id', 'sample', 2,3,4,'fp',6,7,8,'uuid',10]

In [ ]:
denali_merged = df2.merge(df1, on='sample')

In [ ]:
len(set(denali_merged['sample']))

In [ ]:
set(denali_merged['sample']) - set(merged['sample'])

# pair up fastqs

In [ ]:
import os

In [ ]:
DIR = '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/fastqs/'
fps = [DIR + fp for fp in os.listdir(DIR)]

In [ ]:
tups = []

for fp in fps:
    if '.R1.' in fp:
        for fp_2 in fps:
            if fp_2.replace('.R2.', '.R1.') == fp and fp != fp_2:
                tups.append((fp, fp_2))

In [ ]:
tups[:5]

In [ ]:
len(tups)

In [ ]:
s = ''
for fp1, fp2 in tups:
    s += f'{fp1}\t{fp2}\n'

f = open('/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/paired_fastqs.tsv', 'w')
f.write(s)
f.close()

## create yaml cwl configs

In [2]:
import re
import os

import yaml

In [3]:
threads = 10
compressed_input = True
genome_dir = '/gscmnt/gc2508/dinglab/estorrs/cromwell_sandbox/results/gneiss_generate_genome_dir.cwl/18bd05da-dbc9-4742-a210-8129743e1a2b/call-gneiss_generate_genome_dir.cwl/execution/genome_dir'

In [4]:
DIR = '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/fastqs/'
fps = [DIR + fp for fp in os.listdir(DIR)]

tups = []

for fp in fps:
    if '.R1.' in fp:
        for fp_2 in fps:
            if fp_2.replace('.R2.', '.R1.') == fp and fp != fp_2:
                tups.append((fp, fp_2))
                
len(tups)

432

In [5]:
sample_to_config = {}
for r1, r2 in tups:
    d = {
        'threads': threads,
        'compressed_input': compressed_input,
        'genome_dir': {
            'class': 'Directory',
            'path': genome_dir
        },
        'read_1_fastq': {
            'class': 'File',
            'path': r1
        },
        'read_2_fastq': {
            'class': 'File',
            'path': r2
        }
    }
    
    sample = r1.split('/')[-1]
    sample = re.sub(r'^(.*)\.RNA-Seq\.R.\.(.)\.fastq$', r'\1.\2', sample)
    
    sample_to_config[sample] = d

In [6]:
len(sample_to_config.keys())

432

In [7]:
for k, v in sample_to_config.items():
    with open(f'/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/{k}.alignment.yaml', 'w') as outfile:
        yaml.dump(v, outfile, default_flow_style=False)

## create mgi bsub commands

In [8]:
config_dir = '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/'
cromwell_config = '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/cromwell.config'
log_prefix = '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/logs/'

In [9]:
fps = [config_dir + fp for fp in os.listdir(config_dir)]
len(fps)

432

In [10]:
# sort filepaths. only necessary for test runs
fps = sorted(fps)
fps[:10]

['/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00001.A.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00001.T.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00004.N.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00004.T.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00006.A.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00006.T.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00009.A.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00009.T.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00010.A.alignment.yaml',
 '/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00010.T.alignment.yaml']

In [11]:
bsub_commands = []
for fp in fps:
    sample = fp.split('/')[-1]
    sample = log_prefix + sample.replace('.yaml', '.log')
    command = "bsub -q research-hpc -o " + sample + " -a 'docker(registry.gsc.wustl.edu/apipe-builder/genome_perl_environment:5)' \
/usr/bin/java -Dconfig.file=" + cromwell_config + " -jar /opt/cromwell.jar run -t cwl -i \
" + fp + " /gscmnt/gc2508/dinglab/estorrs/gneiss/cwl/gneiss_align.cwl"
    bsub_commands.append(command)

In [12]:
bsub_commands[0]

"bsub -q research-hpc -o /gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/logs/C3L-00001.A.alignment.log -a 'docker(registry.gsc.wustl.edu/apipe-builder/genome_perl_environment:5)' /usr/bin/java -Dconfig.file=/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/cromwell.config -jar /opt/cromwell.jar run -t cwl -i /gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/configs/C3L-00001.A.alignment.yaml /gscmnt/gc2508/dinglab/estorrs/gneiss/cwl/gneiss_align.cwl"

In [13]:
lines = '\n'.join(bsub_commands) + '\n'

In [14]:
f = open('/gscmnt/gc2508/dinglab/estorrs/rnaediting/mgi/alignment/bsub_commands.sh', 'w')
f.write(lines)
f.close()

### write sample to bam location file

In [44]:
import os
import re

def listfiles(folder, regex=None):
    for root, folders, files in os.walk(folder):
        for filename in folders + files:
            if regex is None:
                yield os.path.join(root, filename)
            elif re.findall(regex, filename):
                yield os.path.join(root, filename)
            

In [45]:
REPO_DIRECTORY = '/gscmnt/gc2508/dinglab/estorrs/rnaediting/'
DATA_DIRECTORY = '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/'

In [48]:
CWL_ALIGNMENT_RESULTS_DIRECTORY = DATA_DIRECTORY + 'cwl/alignment/results/'
OUTPUT_FP = REPO_DIRECTORY + 'mgi/alignment/aligned_bams.test_50.locations.tsv'

In [38]:
bam_fps = listfiles(CWL_ALIGNMENT_RESULTS_DIRECTORY, regex=r'.bam$')
bam_fps = list(bam_fps)
len(bam_fps), bam_fps[:5]

(50,
 ['/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/79296761-5673-4bb0-b9e9-b32ef117a857/call-gneiss_align.cwl/execution/gneiss_outputs/Aligned.sortedByCoord.out.bam',
  '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/3c15f612-4db6-4c96-b067-d498367ab4bc/call-gneiss_align.cwl/execution/gneiss_outputs/Aligned.sortedByCoord.out.bam',
  '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/a8981a4e-14be-4bf7-9226-ebea7178a329/call-gneiss_align.cwl/execution/gneiss_outputs/Aligned.sortedByCoord.out.bam',
  '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/63d5603f-9e02-4fac-903c-d2edcd7de956/call-gneiss_align.cwl/execution/gneiss_outputs/Aligned.sortedByCoord.out.bam',
  '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/4e7fc0be-cb15-47eb-b333-28a3acf426d4/call-gneiss_align.cwl/execution/gneiss_outputs/Al

In [39]:
fastq_input_fps = listfiles(CWL_ALIGNMENT_RESULTS_DIRECTORY, regex=r'.fastq$')
fastq_input_fps = list(fastq_input_fps)

sample_to_sub_directory = {}
for fp in fastq_input_fps:
    sample = re.sub(r'^.*/(.*).RNA-Seq.R[0-9].(.).fastq', r'\1.\2', fp)
    sub_directory = re.sub(r'(^.*)call-gneiss_align.*$', r'\1', fp)
    
    sample_to_sub_directory[sample] = sub_directory
    
len(sample_to_sub_directory), [(k, v) for k, v in sample_to_sub_directory.items()][:5]

(50,
 [('C3L-00279.A',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/79296761-5673-4bb0-b9e9-b32ef117a857/'),
  ('C3L-00279.T',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/3c15f612-4db6-4c96-b067-d498367ab4bc/'),
  ('C3L-00144.T',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/a8981a4e-14be-4bf7-9226-ebea7178a329/'),
  ('C3L-00140.A',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/63d5603f-9e02-4fac-903c-d2edcd7de956/'),
  ('C3L-00017.T',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/4e7fc0be-cb15-47eb-b333-28a3acf426d4/')])

In [42]:
sample_fp_tups = []
for sample, sub_dir in sample_to_sub_directory.items():
    for bam_fp in bam_fps:
        if sub_dir in bam_fp:
            sample_fp_tups.append((sample, bam_fp))
sample_fp_tups = sorted(sample_fp_tups, key=lambda x: x[0])
            
len(sample_fp_tups), sample_fp_tups[:5]

(50,
 [('C3L-00001.A',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/a5348b79-eb38-4a32-8626-43fa8d45f88e/call-gneiss_align.cwl/execution/gneiss_outputs/Aligned.sortedByCoord.out.bam'),
  ('C3L-00001.T',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/5ef69396-3817-4903-9c16-4a8d59ef5506/call-gneiss_align.cwl/execution/gneiss_outputs/Aligned.sortedByCoord.out.bam'),
  ('C3L-00004.N',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/b4e11ef1-c64b-4061-8ce1-662b4304d696/call-gneiss_align.cwl/execution/gneiss_outputs/Aligned.sortedByCoord.out.bam'),
  ('C3L-00004.T',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gneiss_align.cwl/a3dcdffc-a5b6-4527-8b8f-61be706b9e61/call-gneiss_align.cwl/execution/gneiss_outputs/Aligned.sortedByCoord.out.bam'),
  ('C3L-00006.A',
   '/gscmnt/gc2508/dinglab/estorrs/data/rnaediting/cwl/alignment/results/gnei

In [49]:
output_str = ''
for sample, fp in sample_fp_tups:
    output_str += f'{sample}\t{fp}\n'

f = open(OUTPUT_FP, 'w')
f.write(output_str)
f.close()